In [6]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [8]:
# Creating tokenizer through a pretrained BERT model from huggingface
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\Owner\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
tokens = tokenizer.encode('I dont know how I felt about it. It was just okay.', return_tensors='pt')

In [19]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-0.9476,  1.5553,  2.7927, -0.2111, -2.7610]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [20]:
#Sentiment score of the text encoded in tokens above
sentiment_score = int(torch.argmax(result.logits)) + 1
sentiment_score

3

In [22]:
r = requests.get('https://www.yelp.com/biz/artichoke-basilles-pizza-berkeley?osq=Artichoke+Basille%27s+Pizza')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [26]:
reviews[0]

"Artichoke Pizza in Berkeley is a hidden gem for pizza lovers, especially those seeking a no-frills, late-night option. This unassuming joint is open late, making it a go-to spot for night owls and students needing a break from studying. The first thing you'll notice is the size of the slices - they're massive and truly bang for your buck.The standout here is the artichoke flavor. It's a delightful blend of creamy and savory, with the perfect balance of artichoke hearts, spinach, and a rich, cheesy sauce. Each bite is a burst of flavor that's hard to beat. What makes it even better is that they heat each slice for you, ensuring it's served hot and fresh, just the way pizza should be.Despite its no-frills appearance, the quality of the pizza at Artichoke Pizza is top-notch. The crust is just right - not too thick, not too thin - and has that perfect crunch. The staff is friendly and efficient, making sure you get your slice quickly even during the busiest hours.If you're in Berkeley and

In [27]:
df = pd.DataFrame(np.array(reviews), columns=['reviews'])

In [31]:
df.head()

,reviews
0,Artichoke Pizza in Berkeley is a hidden gem fo...
1,Really miss the NYC Artichoke pizza and this i...
2,"So, so good!!I got the artichoke pizza and my ..."
3,A little staple and gem in Berkeley! My friend...
4,1000/10 artichoke pizza!!! I visit my bf and m...


In [33]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

In [34]:
sentiment_score(df['reviews'].iloc[1])

4

In [35]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [36]:
df

,reviews,sentiment
0,Artichoke Pizza in Berkeley is a hidden gem fo...,4
1,Really miss the NYC Artichoke pizza and this i...,3
2,"So, so good!!I got the artichoke pizza and my ...",5
3,A little staple and gem in Berkeley! My friend...,5
4,1000/10 artichoke pizza!!! I visit my bf and m...,5
5,YOU'VE GOT TO TRY! Super unique pizza spot tha...,5
6,"So, so good!! I got the artichoke pizza and my...",5
7,It was my first time to the well-loved Articho...,5
8,The service here is nice and quick. I ordered ...,4
9,"8:30pm on a Monday night, it wasn't very busy....",3


In [39]:
df['reviews'][8]

'The service here is nice and quick. I ordered a slice of Margherita and a Vodka Sicilian, both of which had a crispy undercarriage and good sauce. I thought the Sicilian could have had a bit more depth in terms of texture and chew, but overall not too much to complain about.  This is a chill corner spot to grab a quick slice here in Berkeley.'